In [31]:
# animacion_seguidor.py
import math, os
from dataclasses import dataclass
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo  # Python 3.9+
from pathlib import Path

import numpy as np
import pandas as pd
# ⬇️ Pon estas 3 líneas ANTES de importar pyplot
import matplotlib
matplotlib.use("Agg")      # backend sin ventana (evita draw interactivo)
import matplotlib.pyplot as plt
plt.ioff()                 # apaga modo interactivo

from matplotlib.animation import FuncAnimation, FFMpegWriter, PillowWriter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter, PillowWriter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [32]:
# ========= CONFIGURA AQUÍ =========
OUTDIR = Path(r"C:\Users\Matthew\Desktop\Metodos\SeguidorSolar\salidas")
LAT, LON = -0.2105367, -78.491614        # EPN
TZ = ZoneInfo("America/Guayaquil")
FECHA = datetime.now(TZ).date()          # día actual; puedes poner date(2025, 9, 23)
PASO_MIN = 2                              # resolución (minutos) para la animación
DUR_MS = 40                               # ms por frame (velocidad del video)
# ===================================

OUTDIR.mkdir(parents=True, exist_ok=True)

def d2r(d): return d*math.pi/180.0
def r2d(r): return r*180.0/math.pi
def clamp(x,a,b): return max(a, min(b, x))

In [33]:
# --- Posición solar (algoritmo NOAA, sin internet) ---
def solar_position_noaa(dt_local, lat_deg, lon_deg):
    dt_utc = dt_local.astimezone(ZoneInfo("UTC"))
    Y, M, D = dt_utc.year, dt_utc.month, dt_utc.day
    h = dt_utc.hour + dt_utc.minute/60 + dt_utc.second/3600 + dt_utc.microsecond/3.6e9
    if M <= 2:
        Y -= 1; M += 12
    A = math.floor(Y/100)
    B = 2 - A + math.floor(A/4)
    JD = math.floor(365.25*(Y+4716)) + math.floor(30.6001*(M+1)) + D + B - 1524.5 + h/24.0
    T = (JD - 2451545.0)/36525.0

    L0 = (280.46646 + T*(36000.76983 + T*0.0003032)) % 360.0
    M_ = 357.52911 + T*(35999.05029 - 0.0001537*T)
    e = 0.016708634 - T*(0.000042037 + 0.0000001267*T)
    C = (1.914602 - T*(0.004817 + 0.000014*T))*math.sin(d2r(M_)) \
        + (0.019993 - 0.000101*T)*math.sin(d2r(2*M_)) \
        + 0.000289*math.sin(d2r(3*M_))
    true_long = L0 + C
    omega = 125.04 - 1934.136*T
    lambda_app = true_long - 0.00569 - 0.00478*math.sin(d2r(omega))

    eps0 = 23 + (26 + (21.448 - T*(46.815 + T*(0.00059 - 0.001813*T)))/60.0)/60.0
    eps = eps0 + 0.00256*math.cos(d2r(omega))

    sin_delta = math.sin(d2r(eps))*math.sin(d2r(lambda_app))
    delta = math.asin(sin_delta)

    y = math.tan(d2r(eps/2))**2
    EoT = 4*r2d(y*math.sin(2*d2r(L0)) - 2*e*math.sin(d2r(M_)) + 4*e*y*math.sin(d2r(M_))*math.cos(2*d2r(L0))
         - 0.5*y*y*math.sin(4*d2r(L0)) - 1.25*e*e*math.sin(2*d2r(M_)))

    tz_offset_min = dt_local.utcoffset().total_seconds()/60.0
    TST = (dt_local.hour*60 + dt_local.minute + dt_local.second/60.0 + EoT + 4*lon_deg - tz_offset_min) % 1440.0
    HA = TST/4.0 - 180.0
    if HA < -180: HA += 360.0

    lat = d2r(lat_deg); ha = d2r(HA)
    cos_zenith = math.sin(lat)*math.sin(delta) + math.cos(lat)*math.cos(delta)*math.cos(ha)
    cos_zenith = clamp(cos_zenith, -1.0, 1.0)
    zenith = math.acos(cos_zenith)
    elev = 90.0 - r2d(zenith)

    sin_az = math.sin(ha)*math.cos(delta)
    cos_az = math.cos(ha)*math.cos(delta)*math.sin(lat) - math.sin(delta)*math.cos(lat)
    az = (r2d(math.atan2(sin_az, cos_az)) + 360.0) % 360.0
    return az, elev

In [34]:
# --- Ángulos de control (pitch, roll) ---
def pitch_roll_from_az_el(alpha_deg, elev_deg):
    a = d2r(alpha_deg); t = d2r(elev_deg)
    pitch = math.asin(clamp(math.cos(t)*math.cos(a), -1.0, 1.0))
    roll  = math.atan2(math.cos(t)*math.sin(a), math.sin(t))
    return r2d(pitch), r2d(roll)

In [35]:
from datetime import datetime, timedelta
from pytz import timezone

# Pedir fecha y hora de inicio
fecha_str = input("Ingrese la fecha y hora de inicio (YYYY-MM-DD HH:MM): ")
fecha_inicio = datetime.strptime(fecha_str, "%Y-%m-%d %H:%M")

# Pedir fecha y hora final
fecha_fin_str = input("Ingrese la fecha y hora final (YYYY-MM-DD HH:MM): ")
fecha_fin = datetime.strptime(fecha_fin_str, "%Y-%m-%d %H:%M")

# Calcular duración en minutos
duracion_min = int((fecha_fin - fecha_inicio).total_seconds() / 60)

# Paso en minutos
paso_min = 10  # puedes ajustarlo

LAT, LON = -0.2105367, -78.491614
TZ = timezone("America/Guayaquil")
t = TZ.localize(fecha_inicio)

rows = []
for _ in range(0, duracion_min, paso_min):
    az, el = solar_position_noaa(t, LAT, LON)
    if el > 0:
        pitch, roll = pitch_roll_from_az_el(az, el)
    else:
        pitch, roll = None, None
    rows.append({
        "datetime_local": t,
        "azimuth_deg": az,
        "elevation_deg": el,
        "pitch_deg": pitch,
        "roll_deg": roll
    })
    t += timedelta(minutes=paso_min)

df = pd.DataFrame(rows)


In [36]:
# Guardar CSV y dos PNG
csv_path = OUTDIR / f"trayectoria_epn_{FECHA}.csv"
df.to_csv(csv_path, index=False)

plt.figure()
plt.plot(df["elevation_deg"])
plt.title(f"Elevación del Sol (θ) - EPN - {FECHA}")
plt.xlabel("Muestras (cada {} min)".format(PASO_MIN))
plt.ylabel("Elevación (°)")
plt.tight_layout()
png_el = OUTDIR / f"elevacion_{FECHA}.png"
plt.savefig(png_el); plt.close()

plt.figure()
plt.plot(df["pitch_deg"], label="pitch (°)")
plt.plot(df["roll_deg"], label="roll (°)")
plt.title(f"Ángulos de control (pitch/roll) - EPN - {FECHA}")
plt.xlabel("Muestras (cada {} min)".format(PASO_MIN))
plt.ylabel("Ángulo (°)")
plt.legend(); plt.tight_layout()
png_pr = OUTDIR / f"pitch_roll_{FECHA}.png"
plt.savefig(png_pr); plt.close()

In [37]:
# -------- Animación 3D panel + vector solar --------
df_day = df[df["pitch_deg"].notnull()].reset_index(drop=True)
if len(df_day) == 0:
    raise SystemExit("El Sol no está sobre el horizonte en esta fecha. Elige otra FECHA.")

def panel_normal(pitch_deg, roll_deg):
    pi = math.radians(pitch_deg)
    ro = math.radians(roll_deg)
    nx = math.sin(ro) * math.cos(pi)  # E
    ny = math.sin(pi)                 # N
    nz = math.cos(ro) * math.cos(pi)  # U
    return np.array([nx, ny, nz])

def solar_vec(alpha_deg, elev_deg):
    a = math.radians(alpha_deg); t = math.radians(elev_deg)
    sx = math.cos(t) * math.sin(a)  # E
    sy = math.cos(t) * math.cos(a)  # N
    sz = math.sin(t)                # U
    return np.array([sx, sy, sz])

def panel_vertices(normal, size=0.6):
    z_axis = np.array([0,0,1])
    if np.allclose(normal, z_axis):
        v1 = np.array([1,0,0])
    else:
        v1 = np.cross(normal, z_axis); v1 /= np.linalg.norm(v1)
    v2 = np.cross(normal, v1); v2 /= np.linalg.norm(v2)
    center = np.array([0,0,0])
    vs = []
    for dx, dy in [(-1,-1),(-1,1),(1,1),(1,-1)]:
        vs.append(center + dx*size*v1 + dy*size*v2)
    return np.array(vs)

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim([-1,1]); ax.set_ylim([-1,1]); ax.set_zlim([0,1.2])
ax.set_xlabel('Este'); ax.set_ylabel('Norte'); ax.set_zlabel('Arriba')
ax.set_title('Seguidor solar EPN')

Text(0.5, 0.92, 'Seguidor solar EPN')

In [38]:
# Dibujos iniciales
# ... después de crear fig y ax
n0 = panel_normal(df_day.loc[0,"pitch_deg"], df_day.loc[0,"roll_deg"])
verts0 = panel_vertices(n0)
panel_poly = Poly3DCollection([verts0], color='orange', alpha=0.75, edgecolor='k')
ax.add_collection3d(panel_poly)

sv0 = solar_vec(df_day.loc[0,"azimuth_deg"], df_day.loc[0,"elevation_deg"])
sun_quiv = ax.quiver(0,0,0, sv0[0], sv0[1], sv0[2], color='gold', length=1.0, normalize=True)

In [ ]:
def actualizar(i):
    ax.cla()
    # Configuración del gráfico 3D
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(0, 1)
    ax.set_xlabel("Este")
    ax.set_ylabel("Norte")
    ax.set_zlabel("Arriba")

    az, el = df["azimuth_deg"].iloc[i], df["elevation_deg"].iloc[i]
    pitch, roll = df["pitch_deg"].iloc[i], df["roll_deg"].iloc[i]

    if pitch is not None and roll is not None:
        normal_panel = panel_normal(pitch, roll)
        vertices = panel_vertices(normal_panel)
        panel = Poly3DCollection([vertices], alpha=0.5, facecolor="blue")
        ax.add_collection3d(panel)

        # Flecha del Sol
        sol_vec = solar_vec(az, el)
        ax.quiver(0, 0, 0, *sol_vec, color="orange", length=1)

        # Esfera amarilla representando el Sol
        ax.scatter(sol_vec[0], sol_vec[1], sol_vec[2], color="yellow", s=200, edgecolors="k")

    ax.set_title(f"{df['datetime_local'].iloc[i].strftime('%Y-%m-%d %H:%M')}")


In [40]:
# Guardado: MP4 si hay ffmpeg, si no GIF
writers = matplotlib.animation.writers
fps = int(1000/max(DUR_MS,1))

if writers.is_available('ffmpeg'):
    out_mp4 = OUTDIR / f"animacion_seguidor_solar_{FECHA}.mp4"
    ani = FuncAnimation(fig, update, frames=len(df_day), interval=DUR_MS)
    ani.save(out_mp4, writer=FFMpegWriter(fps=fps), dpi=150)
    print("Video MP4 guardado en:", out_mp4)
else:
    out_gif = OUTDIR / f"animacion_seguidor_solar_{FECHA}.gif"
    ani = FuncAnimation(fig, update, frames=len(df_day), interval=DUR_MS)
    ani.save(out_gif, writer=PillowWriter(fps=fps))
    print("GIF guardado en:", out_gif)
import webbrowser
webbrowser.open(str(out_gif))

GIF guardado en: C:\Users\Matthew\Desktop\Metodos\SeguidorSolar\salidas\animacion_seguidor_solar_2025-08-09.gif


True